In [2]:
from dask_jobqueue import SLURMCluster

In [3]:
cluster = SLURMCluster(
    cores=24,
    memory="10 GB"
)
cluster.adapt(maximum_jobs=20)

In [4]:
cluster

SLURMCluster(286778b5, 'tcp://10.203.100.101:35251', workers=0, threads=0, memory=0 B)

In [11]:
from dask.distributed import Client

In [12]:
client = Client(cluster)

In [13]:
print(cluster.dashboard_link)

http://10.203.100.101:39673/status


In [ ]:
pip install scikeras>=0.1.8

In [22]:
import numpy as np
from sklearn.cluster import DBSCAN , OPTICS
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

with joblib.parallel_backend('dask'):
    # #############################################################################
    # Generate sample data
    centers = [[1, 1], [-1, -1], [1, -1]]
    X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4,
                                random_state=0)

    X = StandardScaler().fit_transform(X)

    # #############################################################################
    # Compute DBSCAN
    db = OPTICS( n_jobs = -1).fit(X)
    
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)
    print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
    print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
    print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
    print("Adjusted Rand Index: %0.3f"
          % metrics.adjusted_rand_score(labels_true, labels))
    print("Adjusted Mutual Information: %0.3f"
          % metrics.adjusted_mutual_info_score(labels_true, labels))
    print("Silhouette Coefficient: %0.3f"
          % metrics.silhouette_score(X, labels))

    # #############################################################################
    # Plot result
    import matplotlib.pyplot as plt

    # Black removed and is used for noise instead.
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each)
              for each in np.linspace(0, 1, len(unique_labels))]
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = (labels == k)

        xy = X[class_member_mask & core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=14)

        xy = X[class_member_mask & ~core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=6)

    plt.title('Estimated number of clusters: %d' % n_clusters_)
    plt.show()

Estimated number of clusters: 42
Estimated number of noise points: 434
Homogeneity: 0.422
Completeness: 0.208
V-measure: 0.279
Adjusted Rand Index: 0.014
Adjusted Mutual Information: 0.249
Silhouette Coefficient: -0.266


ValueError: operands could not be broadcast together with shapes (750,) (1750,) 